In [ ]:
import os
import io
import pyarrow as pa
import pyarrow.dataset as ds
import duckdb
import polars as pl
from memory_profiler import profile 
from datetime import datetime
%load_ext memory_profiler
%load_ext line_profiler

In [ ]:
# data ingest and sort
data = ds.dataset("~/data/trade_taq2019.arrow", format = "arrow")

# construct scanner to scanner
data = ds.Scanner.from_dataset(data)

# connect duckdb to database
con = duckdb.connect()

In [ ]:
def window(data,con):
    
    # window query
    query = "SELECT * FROM data WHERE Symbol = 'AAPL' AND Sale_Condition LIKE '%O%'"
    
    result = con.query(query).df()

    return result

%memit m=window(data,con)
m

In [ ]:
def agg(data,con):
     
    # aggregated timebucket query 
    query = "SELECT ANY_VALUE(Time),SUM(Trade_Volume),Symbol FROM data GROUP BY round(extract('epoch' from Time) / 60), Symbol ORDER BY round(extract('epoch' from Time) / 60)"
    
    result = con.query(query).df()
    
    return result

%memit m=agg(data,con)
m

In [ ]:
def new_agg(data,con):

    #aggregating query
    query = "SELECT Symbol,Exchange,AVG(Trade_Price),AVG(Trade_Volume) FROM data GROUP BY Symbol,Exchange ORDER BY AVG(Trade_Price) DESC"
    
    result = con.query(query).df()
    
    return result

%memit m=new_agg(data,con)
m